In [205]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
%aimport AD_comparison_tools
%aimport AD_predictor_tools
%aimport ADpred_LambertTFs_helper
import protfasta
import matplotlib.pyplot as plt
import seaborn as sns
#import metapredict as meta
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, auc

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.lines import Line2D

sns.set_theme(rc={'figure.figsize':(6,4), 'figure.dpi' : 300})
sns.set_style("white")

from parrot import py_predictor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
adhunter = pd.read_csv("../output/merged_ADHunter_Lambert_TF_predictions.csv", index_col = 0)
tada = pd.read_csv("../data/merged_TADA_Lambert_TF_predictions_0.4.csv", index_col = 0)
adpred = pd.read_csv("../output/predictions/adpred_LambertTF_predictions.csv", index_col = 0)
mechanistic = pd.read_csv("../output/predictions/LambertTFs_s_001_lcc_VP16_lch_006_ucc_-08_uch_CITED2_lcs1_inf_lcs2_000_lcs1_inf_ucs2_inf_comp_WFL_tl_039_ws_001_ps1_Charge_ps2_AllHydros", index_col = 0)
paddle = pd.read_csv("../data/Sanborn_HumanTF_Predictions_BothStrengths_Formatted",index_col=0)
parrot_paddle = pd.read_csv("../output/parrot_paddle_human_TF_pred_intervals.csv", index_col=0)
parrot_erij = pd.read_csv("../output/parrot_erijman_human_TF_pred_intervals.csv")

adhunter = adhunter[["uniprotID", "Start", "End"]]
tada = tada[["uniprotID", "Start", "End"]]
adpred = adpred[["uniprotID", "Start", "End"]]
mechanistic["uniprotID"] = mechanistic["GeneName"].str.split("|").str[1]
mechanistic = mechanistic[["uniprotID", "Start", "End"]]
paddle = paddle[["uniprotID", "Start", "End"]]
parrot_erij = parrot_erij[["uniprotID", "Start", "End"]]
parrot_paddle = parrot_paddle[["uniprotID", "Start", "End"]]

display(adhunter)
display(tada)
display(adpred)
display(mechanistic)
display(paddle)
display(parrot_erij)
display(parrot_paddle)

,uniprotID,Start,End
0,Q96QS3,393,436
1,P31270,0,47
2,P31270,110,159
3,P50221,56,99
4,P57073,245,324
...,...,...,...
1664,P17098,14,72
1665,Q9UJW7,11,73
1666,Q9UJW7,111,176
1667,Q9UJW7,187,256


,uniprotID,Start,End
0,P31270,1,41
1,P31270,108,170
2,P50221,58,98
3,P57073,242,318
4,P57073,370,409
...,...,...,...
2391,P17098,4,88
2392,Q9UJW7,12,113
2393,Q9UJW7,116,182
2394,Q9UJW7,184,247


,uniprotID,Start,End
0,Q96QS3,232,249
1,P57073,270,280
2,O14628,17,32
3,P50549,44,61
4,Q14209,399,436
...,...,...,...
716,Q8NAP8,67,88
717,P48742,279,289
718,Q9UI36,607,617
719,P17098,40,51


,uniprotID,Start,End
0,Q8WXT5,26,75
1,Q96MU6,18,83
2,O43313,630,690
3,O14978,79,120
4,O14978,191,234
...,...,...,...
541,P28698,173,226
542,Q53EQ6,500,550
543,Q07869,1,58
544,Q9H0D2,46,107


,uniprotID,Start,End
0,Q6P9G9,155,235
1,Q04206,418,502
2,Q9Y2G1,0,62
3,P43354,13,77
4,Q9ULD5,278,338
...,...,...,...
597,Q03701,943,1030
598,Q2M1K9,51,136
599,Q96LX8,0,107
600,Q9Y2D1,15,104


,uniprotID,Start,End
0,A1YPR0,61,77
1,A1YPR0,237,255
2,A2RRD8,71,81
3,A6NFI3,137,161
4,A6NFI3,210,237
...,...,...,...
610,Q9Y603,47,57
611,Q9Y692,239,254
612,Q9Y6R6,9,19
613,Q9Y6Y1,746,768


,uniprotID,Start,End
0,A0AVK6,211,267
1,A1YPR0,223,355
2,A4D1E1,1053,1119
3,A6NFI3,114,306
4,A6NI15,0,63
...,...,...,...
1036,Q9Y6Q9,783,850
1037,Q9Y6Q9,1052,1105
1038,Q9Y6Y1,716,827
1039,Q9Y6Y1,965,1018


In [209]:
lambert_TFs = pd.read_csv("../data/LambertTFs.csv", index_col = 0)
lambert_TFs["uniprotID"] = lambert_TFs["GeneName"].str.split("|").str[1]
lambert_TFs["TF_length"] = lambert_TFs["ProteinSeq"].str.len()
lambert_TFs

,GeneName,ProteinSeq,uniprotID,TF_length
0,sp|P23511|NFYA_HUMAN,MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASG...,P23511,347
1,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3,562
2,sp|P31270|HXA11_HUMAN,MDFDERGPCSSNMYLPSCTYYVSGPDFSSLPSFLPQTPSSRPMTYS...,P31270,313
3,sp|P50221|MEOX1_HUMAN,MDPAASSCMRSLQPPAPVWGCLRNPHSEGNGASGLPHYPPTPFSFH...,P50221,254
4,sp|P57073|SOX8_HUMAN,MLDMSEARSQPPCSPSGTASSMSHVEDSDSDAPPSPAGSEGLGRAG...,P57073,446
...,...,...,...,...
1603,sp|P35227|PCGF2_HUMAN,MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRY...,P35227,344
1604,sp|Q9BS34|ZN670_HUMAN,MDSVSFEDVAVAFTQEEWALLDPSQKNLYRDVMQEIFRNLASVGNK...,Q9BS34,389
1605,sp|P17098|ZNF8_HUMAN,MDPEDEGVAGVMSVGPPAARLQEPVTFRDVAVDFTQEEWGQLDPTQ...,P17098,575
1606,sp|Q9UJW7|ZN229_HUMAN,METLTSRHEKRALHSQASAISQDREEKIMSQEPLSFKDVAVVFTEE...,Q9UJW7,825


In [210]:
known_ADs= pd.read_csv("../../AD_predictor_project/Paper_AD_predictor/AD_predictor/output/known_ADs_considering_isoforms.csv")
known_ADs = known_ADs[known_ADs["TileType"] == "TF"]
known_ADs = known_ADs[known_ADs["uniprotID"].isin(lambert_TFs["uniprotID"])]
known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
5,AHR,532,848,P35869,"[['P35869'], ['P35869'], ['P35869']]",ENST00000242057 / nan / ENST00000242057,P35869 / P35869 / P35869,"DelRosso et al. / PMID: 8794892, Soto / DelRos...",TF,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
754,ZXDA,572,699,P98168,"[['P98168'], ['P98168']]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto / R4TA_regions.txt / acti...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
755,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
756,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN
758,c-Jun/AP-1,238,257,P05412,[['P05412']],NaN,P05412,"Choi 2000 list, GSL",TF,ETPPLSPIDMESQERIKAER,NaN


In [211]:
def return_tp_fp(preds_input, knownADs):
    preds = preds_input.copy()
    AD_comparison_tools.add_col_contains_prediction(preds, knownADs, result_col_name = "KnownAD")
    TP_count = sum(preds["KnownAD"])
    FP_count = sum(preds["KnownAD"] == False)
    # print("True positives: " + str(TP_count) + "(" + str(np.round((100 * TP_count/ len(preds)), 1)) + "%)")
    # print("False positives: " + str(FP_count)+ "(" + str(np.round((100 * FP_count/ len(preds)), 1)) + "%)")
    return TP_count, FP_count

In [212]:
TP_count, FP_count = return_tp_fp(adhunter, known_ADs)

# Limit assessment to only predictions on TFs containing at least one known AD.
We limited our assessment of model performance by only looking at TFs that contained at least one entry on the GSL or Soto lists, assuming for a moment that all ADs on these TFs are known. Now the performance is X, which is much better. "

In [9]:
model_dict = {
    "TADA": tada,
    "ADHunter": adhunter,
    "ADPred": adpred,
    "Mechanistic": mechanistic,
    "PADDLE": paddle,
    "PARROT_erij" : parrot_erij,
    "PARROT_PADDLE" : parrot_paddle
} 

def eval_model(model_name):
    preds = model_dict[model_name]
    TF_lim_preds = preds[preds["uniprotID"].isin(known_ADs["uniprotID"])]
    num_preds = len(TF_lim_preds)
    benchmark_count = len(known_ADs)
    TP, FP =  return_tp_fp(preds_input = TF_lim_preds, knownADs = known_ADs)
    return model_name, num_preds, benchmark_count, TP, FP

In [10]:
model_results_df = pd.DataFrame(columns=["Predictor", "Predictions", "Benchmark", "True Positives", "False Positives"])
for model in model_dict.keys():
    model_results_df.loc[len(model_results_df)] = eval_model(model)
model_results_df

,Predictor,Predictions,Benchmark,True Positives,False Positives
0,TADA,840,599,446,394
1,ADHunter,655,599,370,285
2,ADPred,291,599,194,97
3,Mechanistic,218,599,148,70
4,PADDLE,271,599,219,52
5,PARROT_erij,1254,599,617,637
6,PARROT_PADDLE,1292,599,557,735


In [15]:
model_results_df["False Negatives"] = model_results_df["Benchmark"] - model_results_df["True Positives"]
model_results_df["Precision"] = (model_results_df["True Positives"] / (model_results_df["True Positives"] + model_results_df["False Positives"])).round(2)
model_results_df["Recall"] = (model_results_df["True Positives"] / (model_results_df["True Positives"] + model_results_df["False Negatives"])).round(2)
model_results_df["F1 Score"] = (2 * (model_results_df["Precision"] * model_results_df["Recall"]) / (model_results_df["Precision"] + model_results_df["Recall"])).round(2)
model_results_df

,Predictor,Predictions,Benchmark,True Positives,False Positives,False Negatives,Precision,Recall,F1 Score
0,TADA,840,599,446,394,153,0.53,0.74,0.62
1,ADHunter,655,599,370,285,229,0.56,0.62,0.59
2,ADPred,291,599,194,97,405,0.67,0.32,0.43
3,Mechanistic,218,599,148,70,451,0.68,0.25,0.37
4,PADDLE,271,599,219,52,380,0.81,0.37,0.51
5,PARROT_erij,1254,599,617,637,-18,0.49,1.03,0.66
6,PARROT_PADDLE,1292,599,557,735,42,0.43,0.93,0.59


In [16]:
model_results_df.sort_values(by = "F1 Score", ascending=False)

,Predictor,Predictions,Benchmark,True Positives,False Positives,False Negatives,Precision,Recall,F1 Score
5,PARROT_erij,1254,599,617,637,-18,0.49,1.03,0.66
0,TADA,840,599,446,394,153,0.53,0.74,0.62
1,ADHunter,655,599,370,285,229,0.56,0.62,0.59
6,PARROT_PADDLE,1292,599,557,735,42,0.43,0.93,0.59
4,PADDLE,271,599,219,52,380,0.81,0.37,0.51
2,ADPred,291,599,194,97,405,0.67,0.32,0.43
3,Mechanistic,218,599,148,70,451,0.68,0.25,0.37


# Repeat using 40-mer windows

In [213]:
# for df in [adhunter, tada, adpred, mechanistic]:
#     df["uniprotID"] = df["GeneName"].str.split("|").str[1] 
    
adhunter["region_type"] = "adhunter"
tada["region_type"] = "tada"
adpred["region_type"] = "adpred"
mechanistic["region_type"] = "mechanistic"
paddle["region_type"] = "paddle"
parrot_erij["region_type"] = "parrot_erij"
parrot_paddle["region_type"] = "parrot_paddle"

In [214]:
known_ADs = pd.read_csv("../../SFARI/output/known_ADs_considering_isoforms_and_canonical_with_alerasool.csv")
known_ADs = known_ADs[["uniprotID", "Start", "End"]]
known_ADs["region_type"] = "known_AD"
known_ADs

,uniprotID,Start,End,region_type
0,Q6UWZ7,121,200,known_AD
1,P35869,118,126,known_AD
2,P35869,266,268,known_AD
3,P35869,532,848,known_AD
4,O43823,2,81,known_AD
...,...,...,...,...
756,Q14CW9,1,120,known_AD
757,A1L443,361,500,known_AD
758,Q9UHA2,18,77,known_AD
759,Q14140,201,260,known_AD


In [215]:
all_regions = pd.concat([adhunter, tada, adpred, mechanistic, paddle, known_ADs, parrot_erij, parrot_paddle])
all_regions

,uniprotID,Start,End,region_type
0,Q96QS3,393,436,adhunter
1,P31270,0,47,adhunter
2,P31270,110,159,adhunter
3,P50221,56,99,adhunter
4,P57073,245,324,adhunter
...,...,...,...,...
1036,Q9Y6Q9,783,850,parrot_paddle
1037,Q9Y6Q9,1052,1105,parrot_paddle
1038,Q9Y6Y1,716,827,parrot_paddle
1039,Q9Y6Y1,965,1018,parrot_paddle


In [250]:
tiled_lambert = AD_predictor_tools.makeTilingDF("../data/lambert_TFs_10-21-24_with_DBD_coords.fasta", window_size = 40, window_spacing = 10)
tiled_lambert["uniprotID"] = tiled_lambert["GeneName"].str.split("|").str[1]
tiled_lambert

Creating new Tiling DF at  ../data/TilingDFs/lambert_TFs_10-21-24_with_DBD_coords_size_40_space_10_AAs_W,F,Y,M,L,Q.csv
Window Size = 40  and Window spacing = 10
Number of Tiling Windows: 90895


,ProteinWindowSeq,GeneName,StartPosition,EndPosition,W,F,Y,M,L,Q,K,R,D,E,Charge,uniprotID
0,MEPEGRGSLFEDSDLLHAGNPKENDVTAVLLTPGSQELMI,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,0,40,0,1,0,2,6,1,1,1,3,5,-6,A0A087WUV0
1,EDSDLLHAGNPKENDVTAVLLTPGSQELMIRDMAEALTQW,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,10,50,1,0,0,2,6,2,1,1,4,4,-6,A0A087WUV0
2,PKENDVTAVLLTPGSQELMIRDMAEALTQWRQLNSPQGDV,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,20,60,1,0,0,2,5,4,1,2,3,3,-3,A0A087WUV0
3,LTPGSQELMIRDMAEALTQWRQLNSPQGDVPEKPRNLVLL,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,30,70,1,0,0,2,7,4,1,3,2,3,-1,A0A087WUV0
4,RDMAEALTQWRQLNSPQGDVPEKPRNLVLLGLPISTPDVI,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,40,80,1,0,0,1,6,3,1,3,3,2,-1,A0A087WUV0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90890,QKYYRSYKKCGKRRQARRTAVIVQQKLRSSLLTKKQDQAA,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1600,1640,0,0,3,0,3,6,7,6,1,0,12,Q9Y6Y1
90891,GKRRQARRTAVIVQQKLRSSLLTKKQDQAARKIMRFLRRC,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1610,1650,0,1,0,1,4,5,5,9,1,0,13,Q9Y6Y1
90892,VIVQQKLRSSLLTKKQDQAARKIMRFLRRCRHSPLVDHRL,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1620,1660,0,1,0,1,6,4,4,7,2,0,9,Q9Y6Y1
90893,LLTKKQDQAARKIMRFLRRCRHSPLVDHRLYKRSERIEKG,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1630,1670,0,1,1,1,5,2,5,8,2,2,9,Q9Y6Y1


In [251]:
joined = pd.merge(tiled_lambert, all_regions, on = "uniprotID")
joined

,ProteinWindowSeq,GeneName,StartPosition,EndPosition,W,F,Y,M,L,Q,K,R,D,E,Charge,uniprotID,Start,End,region_type
0,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,210,253,adhunter
1,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,207,255,tada
2,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,211,267,parrot_paddle
3,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,0,1,0,1,4,1,5,1,1,4,1,A0AVK6,210,253,adhunter
4,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,0,1,0,1,4,1,5,1,1,4,1,A0AVK6,207,255,tada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624996,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,746,768,parrot_erij
624997,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,1086,1100,parrot_erij
624998,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,716,827,parrot_paddle
624999,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,965,1018,parrot_paddle


In [252]:
def return_overlap(i, df = joined):
    return max(0, min(df["End"].iloc[i], df["EndPosition"].iloc[i]) - max(df["Start"].iloc[i], df["StartPosition"].iloc[i]))

overlaps = []
for i in joined.index:
    overlaps.append(return_overlap(i))

joined["overlap"] = overlaps
joined

,ProteinWindowSeq,GeneName,StartPosition,EndPosition,W,F,Y,M,L,Q,K,R,D,E,Charge,uniprotID,Start,End,region_type,overlap
0,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,210,253,adhunter,0
1,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,207,255,tada,0
2,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,0,2,0,2,4,1,4,1,1,6,-2,A0AVK6,211,267,parrot_paddle,0
3,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,0,1,0,1,4,1,5,1,1,4,1,A0AVK6,210,253,adhunter,0
4,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,0,1,0,1,4,1,5,1,1,4,1,A0AVK6,207,255,tada,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624996,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,746,768,parrot_erij,0
624997,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,1086,1100,parrot_erij,0
624998,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,716,827,parrot_paddle,0
624999,RKIMRFLRRCRHSPLVDHRLYKRSERIEKGQGT,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1640,1673,0,1,1,1,3,1,3,8,1,2,8,Q9Y6Y1,965,1018,parrot_paddle,0


In [253]:
summary_table = pd.pivot_table(joined, 
               values='overlap', 
               index=['GeneName', 'StartPosition', 'EndPosition', 'ProteinWindowSeq'],
              columns = 'region_type', aggfunc="sum").reset_index()
summary_table = summary_table.fillna(0)
summary_table

region_type,GeneName,StartPosition,EndPosition,ProteinWindowSeq,adhunter,adpred,known_AD,mechanistic,paddle,parrot_erij,parrot_paddle,tada
0,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,20,60,PLKESTTANIVLAEIQPDFGPLTTPTKPKEGSQGEPWTPT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,30,70,VLAEIQPDFGPLTTPTKPKEGSQGEPWTPTANLKMLISAV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,40,80,PLTTPTKPKEGSQGEPWTPTANLKMLISAVSPEIRNRDQK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
81884,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1600,1640,QKYYRSYKKCGKRRQARRTAVIVQQKLRSSLLTKKQDQAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81885,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1610,1650,GKRRQARRTAVIVQQKLRSSLLTKKQDQAARKIMRFLRRC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81886,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1620,1660,VIVQQKLRSSLLTKKQDQAARKIMRFLRRCRHSPLVDHRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81887,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1630,1670,LLTKKQDQAARKIMRFLRRCRHSPLVDHRLYKRSERIEKG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
summary_table["uniprotID"] = summary_table["GeneName"].str.split("|").str[1]
summary_table

region_type,GeneName,StartPosition,EndPosition,ProteinWindowSeq,adhunter,adpred,known_AD,mechanistic,paddle,parrot_erij,parrot_paddle,tada,uniprotID
0,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,0,40,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A0AVK6
1,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,10,50,EPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPTKPKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A0AVK6
2,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,20,60,PLKESTTANIVLAEIQPDFGPLTTPTKPKEGSQGEPWTPT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A0AVK6
3,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,30,70,VLAEIQPDFGPLTTPTKPKEGSQGEPWTPTANLKMLISAV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A0AVK6
4,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,40,80,PLTTPTKPKEGSQGEPWTPTANLKMLISAVSPEIRNRDQK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A0AVK6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81884,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1600,1640,QKYYRSYKKCGKRRQARRTAVIVQQKLRSSLLTKKQDQAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q9Y6Y1
81885,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1610,1650,GKRRQARRTAVIVQQKLRSSLLTKKQDQAARKIMRFLRRC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q9Y6Y1
81886,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1620,1660,VIVQQKLRSSLLTKKQDQAARKIMRFLRRCRHSPLVDHRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q9Y6Y1
81887,sp|Q9Y6Y1|CMTA1_HUMAN Calmodulin-binding trans...,1630,1670,LLTKKQDQAARKIMRFLRRCRHSPLVDHRLYKRSERIEKG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q9Y6Y1


In [255]:
# Use overlap of at least 10 for each interval
binarized_summary_table = summary_table[["uniprotID"]]
binarized_summary_table["known_AD_binarized"]  = summary_table["known_AD"] > 9


In [256]:
binarized_summary_table["adhunter_binarized"]  = summary_table["adhunter"] > 9
binarized_summary_table["adpred_binarized"]  = summary_table["adpred"] > 9
binarized_summary_table["mechanistic_binarized"]  = summary_table["mechanistic"] > 9
binarized_summary_table["tada_binarized"]  = summary_table["tada"] > 9
binarized_summary_table["paddle_binarized"]  = summary_table["paddle"] > 9
binarized_summary_table["parrot_erij_binarized"]  = summary_table["parrot_erij"] > 9
binarized_summary_table["parrot_paddle_binarized"]  = summary_table["parrot_paddle"] > 9
binarized_summary_table

region_type,uniprotID,known_AD_binarized,adhunter_binarized,adpred_binarized,mechanistic_binarized,tada_binarized,paddle_binarized,parrot_erij_binarized,parrot_paddle_binarized
0,A0AVK6,False,False,False,False,False,False,False,False
1,A0AVK6,False,False,False,False,False,False,False,False
2,A0AVK6,False,False,False,False,False,False,False,False
3,A0AVK6,False,False,False,False,False,False,False,False
4,A0AVK6,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
81884,Q9Y6Y1,False,False,False,False,False,False,False,False
81885,Q9Y6Y1,False,False,False,False,False,False,False,False
81886,Q9Y6Y1,False,False,False,False,False,False,False,False
81887,Q9Y6Y1,False,False,False,False,False,False,False,False


In [257]:
def return_tp_fp_v2(pred_name, binarized_summary_table):
    tp_count = len(binarized_summary_table[binarized_summary_table["known_AD_binarized"] & binarized_summary_table[pred_name + "_binarized"]])
    fp_count = len(binarized_summary_table[~binarized_summary_table["known_AD_binarized"] & binarized_summary_table[pred_name + "_binarized"]])
    return tp_count, fp_count

In [271]:
model_dict = {
    "tada": tada,
    "adhunter": adhunter,
    "adpred": adpred,
    "mechanistic": mechanistic,
    "paddle": paddle,
    "parrot_erij": parrot_erij,
    "parrot_paddle": parrot_paddle
} 

# def eval_model2(model_name):
#     #preds = model_dict[model_name]
#     TF_lim_preds = binarized_summary_table[binarized_summary_table["uniprotID"].isin(known_ADs["uniprotID"])]
#     print(len(TF_lim_preds))
#     num_preds = sum(TF_lim_preds[model_name + "_binarized"])
#     benchmark_count = sum(TF_lim_preds["known_AD_binarized"])
#     TP, FP = return_tp_fp_v2(model_name, TF_lim_preds)
#     return model_name, num_preds, benchmark_count, TP, FP


def eval_model2(model_name):
    #preds = model_dict[model_name]
    TF_lim_preds = binarized_summary_table[binarized_summary_table["uniprotID"].isin(known_ADs["uniprotID"])]
    print(len(TF_lim_preds))
    num_preds = sum(TF_lim_preds[model_name + "_binarized"])
    positive_benchmark_count = sum(TF_lim_preds["known_AD_binarized"])
    negative_benchmark_count = sum(~TF_lim_preds["known_AD_binarized"])
    TP, FP = return_tp_fp_v2(model_name, TF_lim_preds)
    return model_name, num_preds, positive_benchmark_count, negative_benchmark_count, TP, FP



In [273]:
model_results_df_fixed = pd.DataFrame(columns=["Predictor", "Predictions", "Positive Benchmark", "Negative Benchmark", "True Positives", "False Positives"])
for model in model_dict.keys():
    model_results_df_fixed.loc[len(model_results_df_fixed)] = eval_model2(model)
model_results_df_fixed

25801
25801
25801
25801
25801
25801
25801


,Predictor,Predictions,Positive Benchmark,Negative Benchmark,True Positives,False Positives
0,tada,7006,8088,17713,3543,3463
1,adhunter,5057,8088,17713,2685,2372
2,adpred,1025,8088,17713,695,330
3,mechanistic,1641,8088,17713,1078,563
4,paddle,2443,8088,17713,1816,627
5,parrot_erij,825,8088,17713,542,283
6,parrot_paddle,4433,8088,17713,2729,1704


In [275]:
model_results_df_fixed["False Negatives"] = model_results_df_fixed["Positive Benchmark"] - model_results_df_fixed["True Positives"]
model_results_df_fixed["Precision"] = (model_results_df_fixed["True Positives"] / (model_results_df_fixed["True Positives"] + model_results_df_fixed["False Positives"])).round(2)
model_results_df_fixed["Recall"] = (model_results_df_fixed["True Positives"] / (model_results_df_fixed["True Positives"] + model_results_df_fixed["False Negatives"])).round(2)
model_results_df_fixed["F1 Score"] = (2 * (model_results_df_fixed["Precision"] * model_results_df_fixed["Recall"]) / (model_results_df_fixed["Precision"] + model_results_df_fixed["Recall"])).round(2)
model_results_df_fixed

,Predictor,Predictions,Positive Benchmark,Negative Benchmark,True Positives,False Positives,False Negatives,Precision,Recall,F1 Score
0,tada,7006,8088,17713,3543,3463,4545,0.51,0.44,0.47
1,adhunter,5057,8088,17713,2685,2372,5403,0.53,0.33,0.41
2,adpred,1025,8088,17713,695,330,7393,0.68,0.09,0.16
3,mechanistic,1641,8088,17713,1078,563,7010,0.66,0.13,0.22
4,paddle,2443,8088,17713,1816,627,6272,0.74,0.22,0.34
5,parrot_erij,825,8088,17713,542,283,7546,0.66,0.07,0.13
6,parrot_paddle,4433,8088,17713,2729,1704,5359,0.62,0.34,0.44


In [276]:
model_results_df_fixed.sort_values(by = "F1 Score", ascending = False)

,Predictor,Predictions,Positive Benchmark,Negative Benchmark,True Positives,False Positives,False Negatives,Precision,Recall,F1 Score
0,tada,7006,8088,17713,3543,3463,4545,0.51,0.44,0.47
6,parrot_paddle,4433,8088,17713,2729,1704,5359,0.62,0.34,0.44
1,adhunter,5057,8088,17713,2685,2372,5403,0.53,0.33,0.41
4,paddle,2443,8088,17713,1816,627,6272,0.74,0.22,0.34
3,mechanistic,1641,8088,17713,1078,563,7010,0.66,0.13,0.22
2,adpred,1025,8088,17713,695,330,7393,0.68,0.09,0.16
5,parrot_erij,825,8088,17713,542,283,7546,0.66,0.07,0.13
